## Her skal det være en relevant overskrift.

#### Daplas Python pakke (dapla) og bilioteket pandas importeres.

In [ ]:
import dapla as dp
import pandas as pd

#### Kjører metoden show for å få oversikt over hvilke parquet datasett som er tilrettelagt i tilknytning veilderens lagringsområde i sky.  Oversikt blir lest inn i egen dataframe - df_datsets.
Aktuelt lagringsområde blir lagt inn som parameter (string objekt som vi definerer selv) INNDATA_PATH

In [ ]:
INNDATA_PATH = '/felles/veiledning/pyspark/eksempler/'
df_datasets = dp.show(INNDATA_PATH)


#### df_datsets skrives ut i output vindu. Setter opsjonen max_colwith til 150 slik at vi får se all informasjon i df_datasets i output.
Oversikt over ulike options som kan settes i Pandas:https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html

In [ ]:
pd.options.display.max_colwidth = 150
df_datasets

#### Avleder variabel filname fra variabel path

In [ ]:
df_datasets["filename"]= df_datasets["path"].str[len(INNDATA_PATH):]
df_datasets

#### Selketerer ut rader bestående av de filnavn som vi ønsker å trekke ut for videre bearbeiding. 
Viser hvordan man kan bruke liste (Phyton list) for å gjøre en slik seleksjon.

In [ ]:
filenames = ['areal', 'bnp', 'innbyggerantall/2020']
df_datasets = df_datasets[df_datasets.filename.isin(filenames)]
df_datasets

#### Oppretter dictionaryet dict_df og leser så de tre parquet datsettene i output vindu over inn som dataframes i dictionaryet. 
Filnavnene blir keys i dictionaryet. Avslutningsvis i pargraf skrives key-navnene ut i output .

In [ ]:
dict_df = {}
for rows in df_datasets.itertuples():
    dict_df[rows.filename] = dp.read_pandas(rows.path)
for key in dict_df:
    print(key)

### Skriver ut innhold i dataframene i dictionaryet

#### Areal

In [ ]:
dict_df['areal']

#### BNP

In [ ]:
dict_df['bnp']

#### Innbyggerantall 2020

In [ ]:
dict_df['innbyggerantall/2020']

### Fjerner duplikater og kolonner vi ikke ønsker å ha med videre
Vi ønsker å sette sammen informasjon fra datasettene i dictinaryet til et nytt datasett. Dette vil vi gjøre ved å koble de sammen via Landkode variabel. Vi fjerner eventuelle dublikater (tar med første forekomst) på Landkode og dropper variable som vi ikke trenger/ønsker å ha med på det nye datasettet i innledende steg under.


Areal

In [ ]:
dict_df['areal'] = dict_df['areal'].drop_duplicates(subset="Landkode", keep='first', inplace=False).drop(columns='Kilde')

BNP (bruker liste funksjonalitet)

In [ ]:
liste_drop_kolonner = ['Land', 'Kilde', 'År']
dict_df['bnp'] = dict_df['bnp'].drop_duplicates(subset="Landkode", keep='first', inplace=False).drop(columns=liste_drop_kolonner)

Innbyggerantall

In [ ]:
liste_drop_kolonner = ['Land', 'Kilde', 'År']
dict_df['innbyggerantall/2020'] = dict_df['innbyggerantall/2020'].drop_duplicates(subset="Landkode", keep='first', inplace=False).drop(columns=liste_drop_kolonner)

### Kobler sammen datsettene og etablerer en dataframe vi skal utføre statistisk analyse på. 

Kobler først areal og bnp og oppretter dataframen df_areal_bnp. Vi legger ikke denne dataframen inn i dictionaryet. Det hadde imidlertid enkelt latt seg gjøre ved å skrive f.eks. å bytte ut df_areal_bnp med dict['areal_bnp].

In [ ]:
df_areal_bnp = pd.merge(left=dict_df['areal'], right=dict_df['bnp'], left_on='Landkode', right_on='Landkode')

Kobler derettet df_areal_bnp med innbyggerantall/2020 dataframen (i dictionaryet). Dermed får vi produsert den dataframen som vi ænsker å utføre en nærmere statistk analyse på. 

In [ ]:
df_sammensatt = pd.merge(left=df_areal_bnp, right=dict_df['innbyggerantall/2020'], left_on='Landkode', right_on='Landkode')

Skriver ut innhold i df_sammensatt i output

In [ ]:
df_sammensatt

### Lagrer dataframe som parquet datasett på Dapla
Skriver pandas dataframe til GCS bucket i parquet format
Merknad: Prøvde å skrive til hjemmekatalog under /user. Får feilmelding og beskjed om at jeg ikke har tilgang. Skriver derfor inntil videre datasettet til temporært område 
til skattestatistikk. Dette må det rettes opp i.

In [ ]:
dp.write_pandas(df_sammensatt, '/produkt/skatt/person/temp/mif/df_sammensatt', valuation='INTERNAL', state= 'INPUT')


In [ ]:
dp.show('/produkt/skatt/person/temp/mif/df_sammensatt')